In [1]:
# 03_task3_future_forecast.ipynb

# --- 1. Imports & Setup ---
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# --- 1a. Ensure project root is in sys.path ---
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

# --- 1b. Import model functions ---
from src.models_arima import fit_auto_arima, forecast_arima
from src.models_lstm import prepare_lstm_data, build_lstm

# --- 2. Config ---
DATA_DIR = os.path.join(project_root, "data", "processed")
TICKERS = ["TSLA_processed.csv", "BND_processed.csv", "SPY_processed.csv"]
FORECAST_PERIODS = 30
LSTM_WINDOW = 20
LSTM_EPOCHS = 20
LSTM_BATCH_SIZE = 16

# --- 3. Helper: Load & clean data ---
def load_and_clean_csv(filename, data_dir):
    file_path = os.path.join(data_dir, filename)
    df = pd.read_csv(file_path, skiprows=2, parse_dates=["Date"], index_col="Date")

    # Detect close price column
    close_col_candidates = ["Close", "Adj Close", "close", "adj close", "Adj_Close"]
    found_col = None
    for col in close_col_candidates:
        if col in df.columns:
            found_col = col
            break
    if not found_col:
        raise KeyError(f"No close price column found in {filename}. Available: {list(df.columns)}")

    df = df[[found_col]].dropna()
    df.rename(columns={found_col: filename.replace(".csv", "")}, inplace=True)
    return df

# --- 4. Load all tickers ---
price_data = pd.DataFrame()
for ticker_file in TICKERS:
    df = load_and_clean_csv(ticker_file, DATA_DIR)
    if price_data.empty:
        price_data = df
    else:
        price_data = price_data.join(df, how="outer")

price_data.ffill(inplace=True)
print("Loaded price data:")
print(price_data.head())

# --- 5. ARIMA & LSTM Forecasting ---
forecasts = {}

for ticker in price_data.columns:
    print(f"\n=== Forecasting for {ticker} ===")
    series = price_data[ticker].dropna()

    # --- ARIMA ---
    print("Fitting ARIMA...")
    arima_model = fit_auto_arima(series)
    arima_forecast = forecast_arima(arima_model, FORECAST_PERIODS)

    # --- LSTM ---
    print("Fitting LSTM...")
    scaled_series = (series - series.min()) / (series.max() - series.min())
    X, y = prepare_lstm_data(scaled_series.values, LSTM_WINDOW)
    X = X.reshape((X.shape[0], X.shape[1], 1))

    lstm_model = build_lstm((LSTM_WINDOW, 1))
    lstm_model.fit(X, y, epochs=LSTM_EPOCHS, batch_size=LSTM_BATCH_SIZE, verbose=0)

    last_window = scaled_series.values[-LSTM_WINDOW:]
    lstm_forecast_scaled = []
    input_seq = last_window.copy()

    for _ in range(FORECAST_PERIODS):
        pred = lstm_model.predict(input_seq.reshape((1, LSTM_WINDOW, 1)), verbose=0)
        lstm_forecast_scaled.append(pred[0][0])
        input_seq = np.append(input_seq[1:], pred[0][0])

    lstm_forecast = np.array(lstm_forecast_scaled) * (series.max() - series.min()) + series.min()

    forecasts[ticker] = {"arima": arima_forecast, "lstm": lstm_forecast}

# --- 6. Plot results ---
for ticker in forecasts.keys():
    plt.figure(figsize=(10, 5))
    plt.plot(price_data[ticker], label="Historical", color="black")

    # ARIMA forecast
    arima_index = pd.date_range(start=price_data.index[-1] + pd.Timedelta(days=1),
                                periods=FORECAST_PERIODS, freq="B")
    plt.plot(arima_index, forecasts[ticker]["arima"], label="ARIMA Forecast", color="blue")

    # LSTM forecast
    plt.plot(arima_index, forecasts[ticker]["lstm"], label="LSTM Forecast", color="red")

    plt.title(f"{ticker} Price Forecast")
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.legend()
    plt.grid(True)
    plt.show()


2025-08-14 16:36:29.822233: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-14 16:36:29.832762: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-14 16:36:29.856217: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755178589.895968   24673 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755178589.905952   24673 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755178589.936144   24673 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

FileNotFoundError: [Errno 2] No such file or directory: '/home/ablex/Development/AI/kifiya_week11/Time-Series-Forecasting/data/processed/TSLA.csv'